In [4]:
import tabula

In [5]:
tables = tabula.read_pdf("../../cea-data/installed-capacity-raw.pdf", pages='all', multiple_tables=True, lattice=True)

Got stderr: Jun 04, 2025 2:07:50 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Jun 04, 2025 2:07:54 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Jun 04, 2025 2:07:57 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Jun 04, 2025 2:07:57 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Jun 04, 2025 2:07:57 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jun 04, 2025 2:07:57 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jun 04, 2025 2:07:57 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jun 04, 2025 2:07:57 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jun 04, 2025 2:07:57 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jun 04, 2025 2:07:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jun 04, 2025 2:07:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jun 04, 2025 2:07:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jun 04, 2025 2:08:01 PM org.apache.pdfbox.pdmo

In [6]:
print(len(tables))

1961


In [ ]:
for n,df in enumerate(tables):
	df.to_csv("../../cea-data/cea-table-dump/{:04d}.csv".format(n), index=False)

# Parse tables and extract location data using Bing API

In [1]:
import pandas as pd
import numpy as np
import dateparser
from glob import glob
import re
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim, Bing
import geopy
import geopandas
from shapely.geometry import Point, Polygon
from geopy.extra.rate_limiter import RateLimiter

In [2]:
# set up Bing geolocator to parse random addresses

geopy.geocoders.options.default_timeout = None

bing_key = '<insert your API key here>'
geolocator = Bing(api_key=bing_key)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2)

In [5]:
# A helper function to find Indian state given a lat/lon point

state_shps = geopandas.read_file("../other-files/gadm36_IND_1.shp")

def find_state(lon, lat):
	p = Point(lon, lat)
	pnts = geopandas.GeoDataFrame(geometry=[p,],)
	pnts = pnts.assign(**{key: pnts.within(geom) for key, geom in zip(state_shps.NAME_1,state_shps.geometry)})
	
	in_state = pnts.loc[0][1:].values.astype(bool)
	state_names = pnts.columns[1:].values
	
	if np.sum(in_state):
		return(state_names[in_state][0])
	else:
		return("")

In [6]:
string_in_df = lambda s, df: np.sum([df[col].str.contains(s, flags = re.IGNORECASE).sum() for col in df])
find_col_with_string = lambda s, df: np.where([df[col].str.contains(s, flags = re.IGNORECASE).sum() for col in df])[0][0]


In [7]:
tlist = sorted(glob("../../cea-data/cea-table-dump/*.csv"))

output = {"lon":[],"lat":[],"capacity_MW":[],"installed":[], "district":[], "state":[], "type":[]}

In [ ]:
# Loop through all tables in the file, then parse out useful info on installed capacity and location

for nt,tname in enumerate(tlist):
	
	tcode = int(tname.split("/")[-1].split(".")[0])
	if tcode in range(1293,1301): continue #avoid the weird nested tables for part of Maharashtra
	

	#tname = tlist[87]
	print(nt, tname)

	#skip files that are summary headers
	textfile = open(tname, 'r')
	filetext = textfile.read()
	textfile.close()
	matches = re.findall("summary of RE", filetext, re.IGNORECASE)
	if matches: continue

	try:
		df = pd.read_csv(tname, lineterminator="\n").dropna(how='all').dropna(thresh=4)
	except pd.errors.EmptyDataError:
		continue #skip blank files
	
	
	df = df.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\r',  ' ', regex=True)
	
	
	print(df)	

	#check for and remove summary row, if present
	if df.iloc[-1].str.contains('[Tt]otal', flags = re.IGNORECASE, na=False).any():
		df = df[:-1]
	
	

	#first check the whole table for clues about the energy source

	try:
		if string_in_df('[Ww]ind', df):
			source = "Wind"
		elif string_in_df('[Ss]olar', df):
			source = "Solar"
		elif string_in_df('[Hh]ydro', df):
			source = "Hydro"
		elif string_in_df('[Bb]io[\s\S]?[Mm]ass', df):
			source = "Biomass"
		else:
			source = "other"

	except:continue #this only fails on the horribly messy nested Maharashtra tables, so skip those
	

	print(source)

	#use the fact that all tables contain some capacity info to figure out the structure of table
	iy = find_col_with_string('capacity', df)
	cap_col = df.iloc[:,iy]
	start_index = np.where(cap_col.str.contains('capacity', flags = re.IGNORECASE, na=False))[0][-1]
	#print(cap_col)
	#print(start_index)

	

	new_header = df.iloc[start_index] 
	
	
	df = df[start_index+1:] 
	df.columns = [str(h).lower() for h in new_header]
	
	#manual fix to sort botched tables in some Karnatka data
	if tname in ["../../cea-data/cea-table-dump/{:04d}.csv".format(R) for R in range(98,191)]:	
		df.columns = np.roll(df.columns,-1)

	
	print(df)


	#extract and convert useful data

	cap_raw = np.ravel(df.filter(like='capacity').values)
	capacities = []
	for c in cap_raw:
		if type(c)==str:
			c = re.sub(r'[^0-9\.]', '', c)
			if c:
				if c[0] ==".":
					c = c[1:]
			else:
				capacities.append(np.nan)
				continue
		capacities.append(float(c))
			
	

	date_raw = np.ravel(df.filter(like='date').values)
	dates = [dateparser.parse(d) if type(d)==str else np.nan for d in date_raw]

	if "state" in df.columns:
		states = np.ravel(df.filter(like='state').values)
	else:
		states = np.zeros_like(dates)
		states[:]=""


	locations = np.ravel(df.filter(like='location').values)
	
	#filter out remaining nans
	locations = [L.split("-")[0] if type(L)==str else "" for L in locations]
	states = [S if type(S)==str else states[0] for S in states]

	lstrings = [a.replace("\"","").replace("dist","").replace("Dist","").strip()+", "+b+", India" for a,b in zip(locations,states)]
	print(lstrings)


	
	
	for lstring in lstrings:
		location = geocode(lstring)
		if location is not None:
			lon = location.longitude
			lat = location.latitude
			state = find_state(lon,lat)
			print(lon, lat, state)

		else:
			lon = np.nan
			lat = np.nan
			state = states[0]
		
		output['lon'].append(lon)
		output['lat'].append(lat)
		output['state'].append(state)
	

	types = [source,]*len(capacities)
	
	output['type'].extend(types)
	output['capacity_MW'].extend(capacities)
	output['installed'].extend(dates)
	output['district'].extend(locations)

In [ ]:
df = pd.DataFrame.from_dict(output)
df.to_csv("../../cea-data/parsed-cea-all.csv")

